# Manejo de Errores

### Tracebacks
Es el cuerpo del texto que puede apuntar al origen (y al final) de un error no controlado. Comprender sus componentes hará que seas más eficaz el corregir errores o depurar un programa que no funciona bien.

In [17]:
#Abrir un archivo inexistente.

open("/path/to/mars.jpg")

FileNotFoundError: [Errno 2] No such file or directory: '/path/to/mars.jpg'

El error es FileNotFoundError (el nombre de excepción), lo que significa que el archivo no existe o quizás el directorio correspondiente no existe.

Siguiente ejemplo: (Continuamos en este archivo)

In [18]:
def main():
    open("/path/to/mars.jpg")

if __name__ == '__main__':
    main()

FileNotFoundError: [Errno 2] No such file or directory: '/path/to/mars.jpg'

La función main() usa un asistente de Python que indica al intérprete que ejecute la función cuando se le llama en la terminal. 

La salida menciona que el error se inicia en la línea 5, que incluye la llamada a main(). A continuación, la salida sigue el error a la línea 2 en la llamada de función open(). 
Y, por último, FileNotFoundError notifica de nuevo que el archivo o el directorio no existen.

### Try y Except de los bloques

A veces resulta útil permitir que se puedan generar excepciones para que otros autores de llamadas puedan tratar los errores.

In [2]:
# Controlar la excepción con un bloque try y except:

try: open('config.txt') #Después de try se agrega código que pueda producir una excepción.
except FileNotFoundError: # except más la posible excepción.
    print("Couldn't find the config.txt file!") #código a ejecutar en caso de que se produzca la excepción.

#Evita un seguimiento y sigue informando al usuario sobre el problema.

Couldn't find the config.txt file!


In [19]:
def main():
    try:
        configuration = open('config.txt')
    except FileNotFoundError:
        print("Couldn't find the config.txt file!")


if __name__ == '__main__':
    main()

Couldn't find the config.txt file!


Si necesitas acceder al error asociado a la excepción, debes actualizar la línea except para incluir la palabra clave as. Esta técnica es práctica si una excepción es demasiado genérica y el mensaje de error puede ser útil:

In [22]:
try: open("mars.jpg")
except FileNotFoundError as err:
    print("got a problem trying to read the file:", err)

#as err significa que err se convierte en una variable con el objeto de excepción como valor.
#Después, usa este valor para imprimir el mensaje de error asociado a la excepción

got a problem trying to read the file: [Errno 2] No such file or directory: 'mars.jpg'


Otra razón para usar esta técnica es acceder directamente a los atributos del error. 

Por ejemplo, si detecta una excepción OSError más genérica, que es la excepción primaria de FilenotFoundError y PermissionError, podemos diferenciarlas mediante el atributo .errno:

In [23]:
try: open("config.txt")
except OSError as err:
    if err.errno == 2:
        print("Couldn't find the config.txt file!")
    elif err.errno == 13:
        print("Found config.txt but couldn't read it")

Couldn't find the config.txt file!


Intenta usar siempre la técnica que proporcione la mejor legibilidad para el código y que ayude a mantenerlo en el futuro. A veces es necesario usar código menos legible para ofrecer una mejor experiencia de usuario cuando se produce un error.

### Generación de excepciones.
En función del error, el código puede tomar deciones para resolver, solucionar o ignorar un problema. 

Ejemplo:
Los astronautas limitan su uso de agua a unos 11 litros al día. 
Vamos a crear una función que, con base al número de astronautas, pueda calcular la cantidad de agua quedará después de un día o más:

In [7]:
def water_left(astronauts, water_left, days_left):
    daily_usage = astronauts * 11
    total_usage = daily_usage * days_left
    total_water_left = water_left - total_usage
    return f"Total water left after {days_left} days is: {total_water_left} liters"

#Probemos con cinco astronautas, 100 litros de agua sobrante y dos días:
water_left(5, 100, 2)

'Total water left after 2 days is: -10 liters'

Esto no es muy útil, ya que una carencia en los litros sería un error. Después, el sistema de navegación podría alertar a los astronautas que no habrá suficiente agua para todos en dos días. Si eres un ingeniero(a) que programa el sistema de navegación, podrías generar una excepción en la función water_left() para alertar de la condición de error:

In [10]:
def water_left(astronauts, water_left, days_left):
    daily_usage = astronauts * 11
    total_usage = daily_usage * days_left
    total_water_left = water_left - total_usage
    if total_water_left < 0:
        raise RuntimeError(f"There is not enough water for {astronauts} astronauts after {days_left} days!")
    return f"Total water left after {days_left} days is: {total_water_left} liters"

In [11]:
#Mismos parámetros
water_left(5, 100, 2)

RuntimeError: There is not enough water for 5 astronauts after 2 days!

En el sistema de navegación, el código para señalar la alerta ahora puede usar RuntimeError para generar la alerta.

La función water_left() también se puede actualizar para evitar el paso de tipos no admitidos. Intenta pasar argumentos que no sean enteros para comprobar la salida de error:

In [12]:
water_left("3", "200", None)

TypeError: can't multiply sequence by non-int of type 'NoneType'

Otro ejemplo de Type Error actualizando la función:

In [13]:
def water_left(astronauts, water_left, days_left):
    for argument in [astronauts, water_left, days_left]:
        try:
            # If argument is an int, the following operation will work
            argument / 10
        except TypeError:
            # TypError will be raised only if it isn't the right type 
            # Raise the same exception but with a better error message
            raise TypeError(f"All arguments must be of type int, but received: '{argument}'")
    daily_usage = astronauts * 11
    total_usage = daily_usage * days_left
    total_water_left = water_left - total_usage
    if total_water_left < 0:
        raise RuntimeError(f"There is not enough water for {astronauts} astronauts after {days_left} days!")
    return f"Total water left after {days_left} days is: {total_water_left} liters"

In [16]:
#Parámetros anteriores
water_left(3, 200, None)

TypeError: All arguments must be of type int, but received: 'None'